In [ ]:
# Collection Timeline and Spending Analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sys
sys.path.append('../src')

from dotenv import load_dotenv
load_dotenv('../.env')

from google.cloud import bigquery
from config.bigquery_config import config

client = config.get_client()

# Collection value over time
timeline_query = """
SELECT 
    pc.purchase_date,
    pc.purchase_price,
    dr.title,
    dr.artist,
    pc.condition,
    pc.personal_rating,
    SUM(pc.purchase_price) OVER (ORDER BY pc.purchase_date) as cumulative_value
FROM `vinyl_catalog.personal_collection` pc
JOIN `vinyl_catalog.discogs_releases` dr ON pc.release_id = dr.release_id
ORDER BY pc.purchase_date
"""

timeline_data = client.query(timeline_query).to_dataframe()
print("Collection Timeline:")
print(timeline_data.to_string(index=False))